### Hier Dokumentiert, was in den einzelnen Schritten gemacht wird. 
Die restliche Dokumentation wird über Kommentare in den Codeblöcken gemacht. 

In [2]:
#Die Libraries die gebraucht werden, werden importiert.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy as spacy
from collections import Counter
#Der Datensatz wird als Variable df aus der CSV importiert. 
#In diesem Datensatz werden die Inhalte mit einem Kommata getrennt. 
df = pd.read_csv("/Users/marenabele/Desktop/Data-Storytelling_2.0/Data/ted-talks.csv", sep =",")
df

#https://dash.plotly.com/dash-core-components/dropdown

,title,author,date,views,likes,link
0,Climate action needs new frontline leadership,Ozawa Bineshi Albert,December 2021,404000,12000,https://ted.com/talks/ozawa_bineshi_albert_cli...
1,The dark history of the overthrow of Hawaii,Sydney Iaukea,February 2022,214000,6400,https://ted.com/talks/sydney_iaukea_the_dark_h...
2,How play can spark new ideas for your business,Martin Reeves,September 2021,412000,12000,https://ted.com/talks/martin_reeves_how_play_c...
3,Why is China appointing judges to combat clima...,James K. Thornton,October 2021,427000,12000,https://ted.com/talks/james_k_thornton_why_is_...
4,Cement's carbon problem — and 2 ways to fix it,Mahendra Singhi,October 2021,2400,72,https://ted.com/talks/mahendra_singhi_cement_s...
...,...,...,...,...,...,...
5435,The best stats you've ever seen,Hans Rosling,February 2006,15000000,458000,https://ted.com/talks/hans_rosling_the_best_st...
5436,Do schools kill creativity?,Sir Ken Robinson,February 2006,72000000,2100000,https://ted.com/talks/sir_ken_robinson_do_scho...
5437,Greening the ghetto,Majora Carter,February 2006,2900000,88000,https://ted.com/talks/majora_carter_greening_t...
5438,Simplicity sells,David Pogue,February 2006,2000000,60000,https://ted.com/talks/david_pogue_simplicity_s...


In [3]:
#Wie viele Autoren gibt es?
count_authors = df["author"].nunique()

#Welche sind die Top Autoren?
#df.value_counts("author", ascending = False).to_frame()
#wurde ausgeklammert, da ich es nun als Variable speichere. Um es zu Stylen muss die Variable 
#als Dataset gespeichert werden, deswegen wird die Serie mit .to_frame umgewandelt.

df_common_author = df.value_counts("author", ascending = False).to_frame(name="Anzahl der Videos")

In [5]:
#TOP 8 Autoren durchschnittliche Likes
mask = (df["author"] == "Alex Gendler") | (df["author"] == "Iseult Gillespie") | (df["author"] == "Matt Walker	") | (df["author"] == "Alex Rosenthal") | (df["author"] == "Elizabeth Cox") | (df["author"] == "Emma Bryce") | (df["author"] == "Daniel Finkel") | (df["author"] == "Juan Enriquez") 
df_author_likes = df[mask]

top_8_autoren = ["Alex Gendler", "Iseult Gillespie", "Matt Walker", "Alex Rosenthal", "Elizabeth Cox", "Emma Bryce",
                 "Daniel Finkel", "Juan Enriquez"]

author_agg = dict()

for author in top_8_autoren:
    mask = df["author"] == author
    author_agg[author] = df[mask]["likes"].mean().__round__()
author_agg

#https://stackoverflow.com/questions/20340844/pandas-create-named-columns-in-dataframe-from-dict

df_top8_likes = pd.DataFrame(list(author_agg.items()),
                      columns=["Autor", "Durchschnittliche Likes"])

#df_top8_likes = pd.DataFrame.from_dict(author_agg, orient="index")
df_top8_likes

,Autor,Durchschnittliche Likes
0,Alex Gendler,126467
1,Iseult Gillespie,50330
2,Matt Walker,46528
3,Alex Rosenthal,28940
4,Elizabeth Cox,60000
5,Emma Bryce,82417
6,Daniel Finkel,78364
7,Juan Enriquez,57500


In [16]:
#https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
#https://blog.finxter.com Cheat Sheet zu Dash
import dash_table
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

#JupyterDash damit die Ausgabe auch schon in VisualCode angezeigt werden kann.
#Starten der App
app = JupyterDash(__name__)

#Gibt das Layout der App an. Bzw. welche Komponenten enthalten sein sollen.
app.layout = html.Div([
    html.H1("TED Talk Explorer"),
    dcc.Dropdown(options=top_8_autoren, value="Alex Gendler", id="dropdown", multi=True),
    dcc.Graph(id="graph"),
    html.Div(id="table")
])

#Für Interaktivitäten des Users
#Verbindet die Dash Komponenten mit Grafiken
#Gibt an, wie die Komponenten mit den Grafiken interagieren.
#Werden aufgerufen, wenn ein Input sich ändert
#Output und Input werden festgelegt
@app.callback(
    Output('table', 'children'),
    [Input("dropdown", "value")]
)
#Updaten des Tables je nach Input-Auswahl
def update_table(dropdown):
    if isinstance(dropdown, str):
        dropdown = [dropdown]
    mask = [a in dropdown for a in df["author"]]
    df_table = df[mask]
    return dash_table.DataTable(df_table.to_dict("records"), [{"name": i, "id": i} for i in df_table.columns])


#Update callback zum Updaten der Grafik
@app.callback(
    Output('graph', 'figure'),
    [Input("dropdown", "value")]
)
def update_figure(dropdown):
    print(dropdown)
    return px.bar(df_top8_likes, x="Durchschnittliche Likes", y="Autor")
#Ausführen der App und Anzeigen der resultate
app.run_server(mode= "inline")

Alex Gendler
Alex Gendler
